In [1]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import math
import random
import statistics
import importlib

import math_expressions as mexpr
importlib.reload(mexpr)

import methods
importlib.reload(methods)

<module 'methods' from '/Users/preetkaria/Desktop/RA/time_to_leave/methods.py'>

In [2]:
alpha_range = [2, 4, 6, 8]
beta_range = [1, 2, 4, 6]
h_range = [1/15, 2/15, 10/15]
c_range = [20, 25, 30, 10]
N_range = range(3, 6)
n_range = [5]

In [3]:
columns = ['alpha', 'beta', 'N', 'n', 'h', 'c', 'mean_n', 'std_n', 'alpha_hat', 'beta_hat', 'u_star', 'u_star_hat', 'z', 'u', 'optimal_cost', 'actual_cost', 'intervals_str']
df = pd.DataFrame(columns=columns)

In [4]:
for _ in range(5):
    alpha = random.choice(alpha_range)
    beta = random.choice(beta_range)
    h = random.choice(h_range)
    c = random.choice(c_range)
    N = random.choice(N_range)
    n = random.choice(n_range)
    intervals = np.random.gamma(shape=alpha, scale=beta, size=N+n)

    mean_n = statistics.mean(intervals[:n])
    std_n = statistics.stdev(intervals[:n])

    alpha_hat, beta_hat = methods.gamma_estimate_parameters(n, intervals)
    alpha_hat = round(alpha_hat) # fractional alpha has no solution for u*, hencing rounding it.

    u = methods.cal_actual_time(n, intervals)
    u_star = methods.get_u_star(N, alpha, beta, h, c)
    u_star_hat = methods.get_u_star(N, alpha_hat, beta_hat, h, c)
    z=u_star / u_star_hat

    optimal_cost = methods.cal_cost(c, h, u, u_star)
    actual_cost = methods.cal_cost(c, h, u, u_star_hat)

    intervals_str = '_'.join([str(x) for x in intervals])
    dic = {'alpha': alpha,
           'beta': beta,
           'N': N,
           'n':n,
           'h': h,
           'c': c,
           'mean_n': mean_n,
           'std_n': std_n,
           'alpha_hat': alpha_hat,
           'beta_hat': beta_hat,
           'u_star': u_star,
           'u_star_hat': u_star_hat,
           'z': z,
           'u': u,
           'optimal_cost': optimal_cost,
           'actual_cost': actual_cost,
           'intervals_str': intervals_str}
    print(dic)
    df.loc[len(df)] = dic


{'alpha': 6, 'beta': 2, 'N': 4, 'n': 5, 'h': 0.6666666666666666, 'c': 20, 'mean_n': 12.443550732949118, 'std_n': 5.159124123553761, 'alpha_hat': 6, 'beta_hat': 2.1389844662067965, 'u_star': 38.09338185787629, 'u_star_hat': 41.27544607794227, 'z': 0.922906606168298, 'u': 27.047200355137484, 'optimal_cost': 20, 'actual_cost': 20, 'intervals_str': '19.330937006565275_6.304903371950173_10.358245578413795_10.274100291961883_15.949567415854462_8.592285681332493_5.864235490403243_6.4768026083394625_6.113876575062283'}
{'alpha': 2, 'beta': 4, 'N': 4, 'n': 5, 'h': 0.06666666666666667, 'c': 30, 'mean_n': 8.348155299244908, 'std_n': 2.4766219835620245, 'alpha_hat': 11, 'beta_hat': 0.7347319532996094, 'u_star': 9.748365611869845, 'u_star_hat': 20.340699601384014, 'z': 0.47925419493469884, 'u': 38.17392146482595, 'optimal_cost': 1.8950370568637405, 'actual_cost': 1.188881457562796, 'intervals_str': '6.42533611970204_7.267416045547929_12.118881589280708_6.3588402520829925_9.570302489610869_5.0359709

In [5]:
display(df.head())
methods.plot_plotly(data=df['actual_cost'], data_label='Cost')

,alpha,beta,N,n,h,c,mean_n,std_n,alpha_hat,beta_hat,u_star,u_star_hat,z,u,optimal_cost,actual_cost,intervals_str
0,6,2,4,5,0.666667,20,12.443551,5.159124,6,2.138984,38.093382,41.275446,0.922907,27.047200,20.000000,20.000000,19.330937006565275_6.304903371950173_10.358245...
1,2,4,4,5,0.066667,30,8.348155,2.476622,11,0.734732,9.748366,20.340700,0.479254,38.173921,1.895037,1.188881,6.42533611970204_7.267416045547929_12.11888158...
2,8,6,3,5,0.666667,10,48.678803,16.972756,8,5.917862,216.726649,211.572577,1.024361,111.995762,10.000000,10.000000,20.79192862252269_56.65662483403159_64.4600453...
3,8,6,5,5,0.666667,20,39.532671,11.340199,12,3.253009,269.480306,195.829568,1.376096,228.044282,20.000000,21.476476,23.400532927359993_39.20904124992205_35.616708...
4,4,6,5,5,0.133333,30,21.104856,10.097073,4,4.830684,77.248992,60.195813,1.283295,123.206460,6.127662,8.401420,29.600895845585907_4.3327499423800635_23.08522...


In [6]:
df.to_csv('../data/gen_data.csv', index=False)